In [1]:
import pandas as pd
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, HfFolder
import os

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# Set the token for use in the huggingface_hub library
HfFolder.save_token(hf_token)

import wandb
from datasets import Dataset

# Replace 'your-api-token' with your actual API token from wandb.ai
wandb.login(key='88bc0f92c0138587605140040c6d0ef652bde63d')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
df = pd.read_csv("/kaggle/input/nlp-mental-health-conversations/train.csv")

In [3]:
df = df.dropna()

In [4]:
df

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


In [5]:
df = df.sample(frac=1).reset_index(drop=True)

In [6]:
df

,Context,Response
0,"I have no friends, no hobbies, and no interest...","Hello, and thank you for your question. It cer..."
1,I had a dispute with my therapist regarding an...,My recommendation would be to try to talk to y...
2,"I'm in my early 20s, and I've been seeing my b...",My first thought was that I wondered what chan...
3,I have been married for 11 years. Within the p...,I'm sorry for how you're feeling in your marri...
4,"I've been suppressing it for quite some time, ...","Human attractions can be tricky things, and in..."
...,...,...
3503,"In middle school and high school, my friends a...",Use this time to explore who you are...imagine...
3504,My ex-wife married and used me to have a child...,The thing that confuses a child the most is fo...
3505,I’ve been on 0.5 mg of Xanax twice a day for t...,Staying on the lower dose may give you more ro...
3506,I've always thought that there wasn't much goo...,Congrats on having a happiness problem.Go slow...


In [7]:
data = df.to_dict(orient='list')
#data
data = Dataset.from_dict(data)

In [8]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:171: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
def preprocess_function(examples):
    # Tokenize the contexts. No need for return_tensors here.
    model_inputs = tokenizer(examples["Context"], max_length=512, truncation=True, padding="max_length")

    # Tokenize the responses. Note: We don't use return_tensors here to avoid the error.
    labels = tokenizer(examples["Response"], max_length=512, truncation=True, padding="max_length")

    # Here's a crucial part: setting -100 for the padding token ids in labels so they're not used in loss calculation
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_example] 
        for label_example in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_data = data.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [11]:
%pip install evaluate
%pip install rouge_score
%pip install bert-score  # If using BERTScore

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 995.0 kB/s eta 0:00:000:00:010m
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8e97b518f05aa441f673a4f4be1eb11016d6cdcff92c57046152164148a2bd8e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 862.6 kB/s eta 0:00:00--:--
Note: you may need to restart the kernel to use updated packages.


In [12]:
#!pip install -U nltk
#from nltk.translate.meteor_score import meteor_score
import evaluate

'''metrics = evaluate.combine(
    [
        evaluate.load("rouge", use_aggregator=False),
        #evaluate.load("meteor"),  # Adding METEOR
        evaluate.load("bertscore", lang='en'),  # Adding BERTScore
    ]
)'''
metrics = evaluate.load("rouge")

2024-03-31 04:24:09.428377: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 04:24:09.428502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 04:24:09.590910: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metrics.compute(predictions=decoded_preds, references=decoded_labels)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # Adjust rounding to handle lists and non-numeric values
    def safe_round(value, digits):
        if isinstance(value, (int, float)):
            return round(value, digits)
        elif isinstance(value, list):
            return [safe_round(v, digits) for v in value]
        else:
            return value

    rounded_result = {k: safe_round(v, 4) for k, v in result.items()}

    return rounded_result


In [14]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [15]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
checkpoint = "google-t5/t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
tokenized_data = tokenized_data.train_test_split(test_size=0.25)

In [17]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2631
    })
    test: Dataset({
        features: ['Context', 'Response', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 877
    })
})

In [18]:
print(tokenized_data["train"].features)

{'Context': Value(dtype='string', id=None), 'Response': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="Mental_health_response",
    evaluation_strategy="epoch",  # Change from "epoch" to "steps"
    learning_rate=2e-3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.1,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub()